## Flipping coins over the telephone

By: Alex Comerford (alexanderjcomerford@gmail.com)

In this notebook we will be demonstrating a minimal example of the coin fliping protocol demonstrated and explained here https://www.cs.cmu.edu/~mblum/research/pdf/coin

Using default capabilities in python we will assume both the identities of alice and bob to show that both parties will agree on the result of a coin toss. In this notebook we will demonstrate this commitment scheme and how each party would deny the other if either of them violate the designated protocol.

## Imports

In [137]:
import string
import random as r

## Utility functions

These lambda functions are to provide short hand names for common functions we will use throughout this notebook

In [12]:
flip_coin = lambda:r.randint(0,1)
generate_secret = lambda: ''.join(r.choice(string.ascii_lowercase) for i in range(5))

## Classes

Here we will define some high level classes with methods to make the steps in this commitment scheme protocol more obvious

In [180]:
class Person:
    def __init__(self, name="", log_output=False):
        self.secret = generate_secret()
        self.log_output = log_output
        self.name = name
        if log_output:print("Person %s's secret is %s"%(self.name, self.secret))
            
        self.is_dishonest = flip_coin()
        
        self.knowledge = {}
        
    def change_secret(self):
        self.secret = generate_secret()
        if self.log_output:print("Person %s secret changed to %s"%(self.name, self.secret))
        
    def set_information(self,key,value):
        if self.log_output:print("Person %s setting information %s as %s"%(self.name, str(key), str(value)))        
        self.knowledge[key] = value

    def get_information(self,key):
        if self.log_output:print("Person %s getting information %s -> %s"%(self.name, str(key), self.knowledge[key]))
        return self.knowledge[key]

## Protocol

Alice is the coin flipper
Bob is the guesser

1. Alice and Bob generate (random) keys
2. Bob sends Alice his key
3. Alice secretly flips a coin and sends her commitment to the flip to Bob
4. Bob announces his guess
5. Alice reveals her key and her flip
6. Bob verifies his guess

#### 1. Alice and Bob generate (random) keys

First we will generate alice and bob as objects which add secret attributes on initialization. Each object is unaware of the other

In [193]:
bob = Person(name="bob", log_output=True)
alice = Person(name="alice", log_output=True)

Person bob's secret is jxztk
Person alice's secret is ofkqv


#### 2. Bob sends Alice his key

In [194]:
alice.set_information("bob.secret",bob.secret)

Person alice setting information bob.secret as jxztk


#### 3. Alice secretely flips a coin and sends her commitment to bob

In [195]:
## Alice generates commitment
alice.set_information("flip",flip_coin())
alice.set_information("commitment", hash(str([alice.get_information("flip"),
                                              alice.secret,
                                              alice.get_information("bob.secret")])))

## Alice sends commitment
bob.set_information("alice.commitment", alice.get_information("commitment"))

Person alice setting information flip as 0
Person alice getting information flip -> 0
Person alice getting information bob.secret -> jxztk
Person alice setting information commitment as -687192656363196433
Person alice getting information commitment -> -687192656363196433
Person bob setting information alice.commitment as -687192656363196433


#### 4. Bob announces his guess

In [221]:
## Bob thinks of this guess
bob.set_information("guess", flip_coin())

## Bob announces that guess to alice
alice.set_information("bob.guess", bob.get_information("guess"))

Person bob setting information guess as 0
Person bob getting information guess -> 0
Person alice setting information bob.guess as 0


#### 5. Alice reveals her key and her flip

In [222]:
if alice.is_dishonest:
    ## Alice changes her flip
    alice.set_information("dishonest", alice.is_dishonest)
    alice.set_information("flip", int(not alice.get_information("flip")))
    
    ## Alice announces manipulated flip
    bob.set_information("alice.flip",alice.get_information("flip"))
    bob.set_information("alice.secret",alice.secret)
else:
    ## Alice announces unmanipulated flip
    bob.set_information("alice.flip",alice.get_information("flip"))
    bob.set_information("alice.secret",alice.secret)    

Person alice getting information flip -> 0
Person bob setting information alice.flip as 0
Person bob setting information alice.secret as ofkqv


#### 6. Bob verifies his guess is correct and if alice is a liar

In [223]:
if bob.get_information("alice.commitment") ==\
   hash(str([bob.get_information("guess"), 
             bob.get_information("alice.secret"), 
             bob.secret])):
    print("Bob has won! yay! :)")
else:
    print("Bob has lost :'(")
    
if bob.get_information("alice.commitment") ==\
   hash(str([bob.get_information("alice.flip"), 
             bob.get_information("alice.secret"), 
             bob.secret])):
    print("Alice told the truth!")
else:
    print("Alice lied in her reveal!")

Person bob getting information alice.commitment -> -687192656363196433
Person bob getting information guess -> 0
Person bob getting information alice.secret -> ofkqv
Bob has won! yay! :)
Person bob getting information alice.commitment -> -687192656363196433
Person bob getting information alice.flip -> 0
Person bob getting information alice.secret -> ofkqv
Alice told the truth!
